论文中使用这个方法的步骤大致如下：
1. 清理评论数据，找出10万条评论数据，然后一一进行标注分类。
2. 分类包括：1.想去，2.不想去，3.不明确，4.无关
3. 用BERT进行训练。
4. 训练模型的过程要在论文中进行描述，包括：使用了什么算法、采用了多少数据，最后给出关于这4个类型的混淆矩阵的对比图，以及训练过程中loss的变化图。
5. 后续用这个模型来预测其他的所有的旅游意愿。

# 使用预训练的BERT完成分类任务
- 参考：https://www.bilibili.com/video/BV1KT421S7K4/?spm_id_from=333.337.search-card.all.click&vd_source=2b245db5a324284a2f8c5e57c3fea278
- 这里使用了18万条中文新闻标题数据和对应的类别标签。
- 采用google-bert/bert-base-chinese预训练模型。


In [ ]:
# 安装依赖
# %pip install transformers
%pip install torch

In [ ]:
import os
import torch
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import BertTokenizer
from transformers import BertModel

# 加载预训练模型
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# 打印分词器最大支持的长度，结果是512，也即中文的长度不能超过512
print(f"Maximum length of the sentence: {tokenizer.model_max_length}")
# 定义待处理的文笔
texts=["你好，世界","我喜欢编程。","这是一个测试句子。"]

# 计算分词结果
# 关于Tokenizer的详细用法，参考huggingface官网
tokens=tokenizer(texts,add_special_tokens=True,#显示的添加特殊令牌
                 padding=True,# 对于未达到最大长度的句子，进行填充
                 max_length=128,# 最大长度为128
                 truncation=True,# 对于超过最大长度的句子，进行截断
                 return_tensors="pt")# 返回pytorch的tensor格式
     
# 将结果中的input_ids和attention_mask取出来
input_ids=tokens["input_ids"]
print(f"Shape of Input IDs: {input_ids.shape}")
attention_mask=tokens["attention_mask"]
print(f" Input IDs: {input_ids}")

print(f"Attention Mask: {attention_mask}")
# 打印分词结果的长度，结果是128，也即中文的长度不能超过128
print(f"Length of the input IDs: {len(input_ids[0])}")

# 加载预训练模型
model = BertModel.from_pretrained('bert-base-chinese')
features = model(input_ids=input_ids,attention_mask=attention_mask)
# 使用features.pooler_output作为后续句子线性分类器的特征向量
pooler_output = features.pooler_output

print(f"Shape of the Pooler Output: {pooler_output.shape}")
print(f"Pooler Output: {pooler_output}") 


Bert实现中文分类：
1. BERT分类模型的设计
2. 中文新闻数据的准备和处理
3. BERT分类模型的训练和评估

In [ ]:

from torch import nn
from transformers import BertModel
# BERT分类模型的类,用于封装BERT模型和线性层
class BertClassifier(nn.Module):
    # 传入预训练的模型路径和分类的类别数量
    def __init__(self, model_path, class_num):
        super(BertClassifier, self).__init__()
        # 加载预训练的BERT模型
        self.bert = BertModel.from_pretrained(model_path)
        # 定义一个线性层，将BERT模型的输出映射到分类的类别数量
        self.classifier=nn.Linear(self.bert.config.hidden_size, class_num)
    # 模型的前向传播
    def forward(self, input_ids, attention_mask):
        # 传入输入的ids和attention_mask，得到BERT模型的输出
        features = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # 取出线性层结果
        logits = self.classifier(features.pooler_output)
        return logits
# 打印模型分类器可以看到相关输出    
# classifier=BertClassifier('bert-base-chinese',10)
# print(classifier)

从Kaggle下载中文新闻分类数据
https://www.kaggle.com/datasets/explorer1024/bert-dataset


In [ ]:
%pip install kagglehub

In [9]:
import kagglehub

# Download latest version 运行下面的代码从网上下载中文新闻数据集，并保存到当前目录的chinese-data目录下
path = kagglehub.dataset_download("explorer1024/bert-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.55G/2.55G [03:26<00:00, 13.2MB/s]

Extracting files...


Path to dataset files: C:\Users\wafd\.cache\kagglehub\datasets\explorer1024\bert-dataset\versions\1


In [10]:
import torch 
from torch.utils.data import Dataset
# 构造数据集
class TextClassifyDataset(Dataset):
    def __init__(self,path):
        self.examples=list()
        file=open(path,'r',encoding='utf-8')
        for line in file:
            text,label=line.strip().split('\t')
            self.examples.append((text,int(label)))
        file.close()
    def __len__(self):
        return len(self.examples)
    def __getitem__(self,idx):
        return self.examples[idx]
# 用于dataloader,对于每个小批量数据，进行分词和填充
def collate_fn(batch,tokenizer):
    texts=[item[0] for item in batch]
    labels=[item[1] for item in batch]
    labels=torch.tensor(labels,dtype=torch.long)
    tokens=tokenizer(texts,return_tensors='pt',padding=True,truncation=True,max_length=512,add_special_tokens=True)
    return tokens['input_ids'],tokens['attention_mask'],labels

接下来开始训练模型

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import torch
from torch import optim
from torch import nn 
import os 

if __name__ == '__main__':
    model_path="bert-base-chinese"

    dataset=TextClassifyDataset(model_path) # 构造数据集
    tokenizer=BertTokenizer.from_pretrained("bert-base-chinese") # 构造分词器

    dataloader=DataLoader(dataset,batch_size=128,shuffle=True,collate_fn=lambda x:collate_fn(x,tokenizer))

    # 读取数据后，定义设备对象
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    # 定义BertClassifier模型
    model=BertClassifier(model_path,class_num=10).to(device)
    model.train()
    optimizer=optim.AdamW(model.parameters(),lr=1e-5) # 优化器
    criterion=nn.CrossEntropyLoss() # 损失函数

    # 创建保存模型的目录
    os.makedirs("output_models",exist_ok=True)
    epoch_n=10
    for epoch in range(1,epoch_n+1): #进入模型的循环迭代
        for batch_id,data in enumerate(dataloader): # 遍历dataloader
            
            input_ids=data[0].to(device)
            attention_mask=data[1].to(device)
            label=data[2].to(device)
            optimizer.zero_grad() # 梯度清零
            output=model(input_ids,attention_mask) #向前传播
            loss=criterion(output,label) # 计算损失
            loss.backward() # 计算梯度
            optimizer.step() # 更新参数

            # 对每组数据，都计算这组数据的正确率，用于观察训练结果
            predict=torch.argmax(output,dim=1)
            correct=(predict==label).sum().item()
            acc=correct/output.size(0)

            print("epoch:{},batch_id:{},loss:{},acc:{}".format(epoch,batch_id,loss.item(),acc))
        # 每一轮迭代，都保存一次模型
        model_name=f"./output_models/chinese_news_classify_epoch_{epoch}.pth"
        print("save model to {}".format(model_name))
        torch.save(model.state_dict(),model_name)



 

    